In [ ]:
# %pip install pandas transformers tokenizers datasets xformers

In [ ]:
!pip install torch==2.2.2 torchvision
!pip install transformers[torch] pandas tokenizers datasets xformers zstandard jsonlines

  Using cached torch-2.2.2-cp310-cp310-manylinux1_x86_64.whl (755.5 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (167.9 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully uninstalled nvidia-nccl-cu12-2.20.5
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [ ]:
dataset = load_dataset("IlyaGusev/ru_turbo_alpaca", split='train')
print(type(dataset))
print(dataset.column_names)
print(type(dataset[0]))
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Ниже будут инструкции для решения задачи, а также текст, содержащий больше контекста. "
        "Напиши отвечающий запросу ответ.\n\n"
        "### Инструкции:\n{instruction}\n\n### Контекст:\n{input}\n\n### Ответ:"
    ),
    "prompt_no_input": (
        "Ниже будут инструкции для решения задачи. "
        "Напиши отвечающий запросу ответ.\n\n"
        "### Инструкции:\n{instruction}\n\n### Ответ:"
    ),
}

In [ ]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [ ]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [ ]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
with open('dataset.json', 'w', encoding='utf-8') as f:
    f.write('[')
    for i, example in enumerate(dataset):
        json_example = json.dumps({
            'instruction': example['instruction'],
            'input': example['input'],
            'output': example['output']
        }, ensure_ascii=False, indent=4)
        if i > 0:
            f.write(',\n')
        f.write(json_example)
    f.write(']')

In [ ]:
model_name = "ai-forever/rugpt3small_based_on_gpt2"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [ ]:
model_name = "ai-forever/rugpt3small_based_on_gpt2"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [ ]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path='dataset.json')
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    learning_rate=1e-5,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    evaluation_strategy='no',
    weight_decay=0.,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    save_strategy='no',
    logging_steps=1000,
    output_dir="rugpt_instruct_ft"
)

In [ ]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

In [ ]:
trainer.train()

Step,Training Loss
1000,2.553300
2000,2.233900
3000,2.183700
4000,2.170500
5000,2.142600
6000,2.087700
7000,2.101700
8000,2.088200
9000,2.070800
10000,2.062900


TrainOutput(global_step=14911, training_loss=2.130407000154039, metrics={'train_runtime': 1899.8368, 'train_samples_per_second': 15.697, 'train_steps_per_second': 7.849, 'total_flos': 2103221219328000.0, 'train_loss': 2.130407000154039, 'epoch': 1.0})

In [ ]:
trainer.save_model('rugpt_instruct_ft')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


In [ ]:
MODEL_NAME = 'rugpt_instruct_ft'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [ ]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Ниже приведено описание задания, сопровождаемое входными данными, которые предоставляют дополнительный контекст. "
              "Напиши ответ, который соответствует запросу.\n\n"
              f"### Описание задания:\n{instruction}\n\n### Входные данные:\n{text}\n\n### Ответ:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
#         max_length=100,
        max_new_tokens=20,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True,
    #     top_k=30,
    #     top_p=0.8,
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [ ]:
text = """
Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов, что привело к длительным отменам по крайней мере на двух линиях.
Во вторник днем все поезда были остановлены на оживленной линии между южными городами Ден Босх и Бокстел после того, как животные зарылись в дамбу, несущую рельсы. Национальная железнодорожная компания сообщила, что линия не будет работать как минимум неделю.
Подкоп означает, что "рельсы могут проседать, и тогда безопасность движения поездов больше не гарантируется", - говорится в заявлении ProRail, компании, обслуживающей железнодорожную сеть Нидерландов.
Ранее в этом месяце барсуки также зарылись под рельсы в районе северной деревни Молкверум в провинции Фрисландия, что вывело линию из эксплуатации до следующего месяца, пока рабочие будут добиваться разрешения на перемещение животных.
Барсуки являются охраняемыми животными в Нидерландах, поэтому железнодорожники должны получить разрешение на их перемещение или нарушение их среды обитания, прежде чем начать ремонт.
"""

In [ ]:
instruction = "Перескажи этот текст."
predict_for_instruction(instruction, text, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов,'

In [ ]:
instruction = "О каком животном идет речь в тексте?"
predict_for_instruction(instruction, text, model)

'Барсуки - это млекопитающее, которое живет в Северной Африке, где они обитают в дикой природе'

In [ ]:
text2 = """

"""

In [ ]:
instruction = "Сколько дней в году?"
predict_for_instruction(instruction, text2, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'В год в среднем по России составляет около 10 дней.'

In [ ]:
instruction = "Придумай описание для книги о кулинарии"
predict_for_instruction(instruction, text2, model)

'Книга "О вкусной и здоровой пище" - это увлекательное и захватывающее путешествие в мир кулина'

In [ ]:
text3 = """Хороша ложка"""

In [ ]:
instruction = "Допиши"
predict_for_instruction(instruction, text3, model)

'Хороша ложка,\nОна полна\nИ вкусна.'